# 예외처리 및 고급 예외

# 예외 처리 정리

---
## 0. 목적
- **코드의 안정성**
- **유지보수**
- **가독성**
- **디버깅 효율**


## 1. 오류의 종류
- **구문 오류 (SyntaxError)**  
  - 문법이 잘못되어 코드가 실행되기 전 단계에서 발생  
- **예외 (Exception)**  
  - 실행 중에 발생하는 문제  
  - 예: 파일이 없을 때, 0으로 나눌 때 등  

---

## 2. 기본 예외 처리
```python
try:
    # 문제가 발생할 가능성이 있는 코드
    x = int(input("숫자 입력: "))
    y = 10 / x
except ZeroDivisionError:
    print("0으로 나눌 수 없어요!")
except ValueError:
    print("숫자로 바꿀 수 없어요!")



자주 쓰이는 예외
```python
ValueError: 잘못된 값(예: int("foo"))

TypeError: 잘못된 자료형(예: len(5))

ZeroDivisionError: 0으로 나눌 때

IndexError / KeyError: 시퀀스나 딕셔너리에 없는 인덱스·키 접근

FileNotFoundError, IOError: 파일 입출력 문제

ImportError, ModuleNotFoundError: 모듈 임포트 실패

Exception : 모든 예외(범용)

## 3. `try … except … else` 구문

```python
try:
    result = fetch_data()   # 외부 API 호출 등
except NetworkError:
    print("네트워크 문제 발생!")
else:
    print("데이터 로드 성공:", result)

In [5]:
# custom_exceptions.py

class NetworkError(Exception):
    """네트워크 관련 오류를 나타내는 예외."""
    def __init__(self, code, message="네트워크 오류 발생"):
        # Exception 메시지를 “[코드] 메시지” 형태로 설정
        super().__init__(f"[{code}] {message}")
        self.code = code
        self.message = message

def fetch_data_from_api():
    """
    외부 API 호출을 흉내내는 함수.
    호출 실패를 가정하고 NetworkError를 발생시킵니다.
    """
    success = False
    status_code = 504  # 예: 게이트웨이 타임아웃
    if not success:
        raise NetworkError(status_code, "API 타임아웃 발생")
    return {"data": "payload"}

try:
    result = fetch_data_from_api()
except NetworkError as e:
    print("⚠️ 예외가 발생했습니다!")
    print("에러 코드:", e.code)        # 504
    print("에러 메시지:", e.message)  # "API 타임아웃 발생"
    print("Exception 인사이더 메시지:", e)  # "[504] API 타임아웃 발생"
else:
    print("데이터 로드 성공:", result)
finally:
    print("네트워크 요청 시도 완료.")


⚠️ 예외가 발생했습니다!
에러 코드: 504
에러 메시지: API 타임아웃 발생
Exception 인사이더 메시지: [504] API 타임아웃 발생
네트워크 요청 시도 완료.


# else 블록은 try 안의 코드가 예외 없이 모두 성공적으로 실행되었을 때만 실행됨

## 4. `finally` 구문

```python
try:
    f = open("data.txt")
    # 파일 읽기 작업 …
except FileNotFoundError:
    print("파일을 못 찾았어요.")
finally:
    f.close()  # 예외가 나든 안 나든 항상 이 줄이 실행돼서 파일이 닫힘



```python
try:
    conn = open_connection()
    # 작업 수행
except:
    print("문제 발생!")
finally:
    conn.close()   # 항상 실행 → 리소스 정리

```python
def process_file(path):
    try:
        # 1) 파일 열기
        with open(path, 'r') as f:
            data = f.read()
        # 2) 문자열을 숫자로 변환
        number = int(data)
        # 3) 0으로 나누기 (의도적 오류)
        result = 100 / number
    except FileNotFoundError as e:
        # 구체적 예외 ①: 파일이 없을 때
        print(f"파일을 찾을 수 없습니다: {e.filename}")
    except ValueError as e:
        # 구체적 예외 ②: 읽어온 내용이 숫자가 아닐 때
        print(f"숫자로 변환할 수 없습니다: {e}")
    except ZeroDivisionError as e:
        # 구체적 예외 ③: 0으로 나눌 때
        print("0으로 나눌 수 없습니다!")
    except Exception as e:
        # 범용 예외: 위에서 처리하지 않은 모든 예외를 한 번에 잡음
        print(f"알 수 없는 오류가 발생했습니다: {e}")
    else:
        # 예외가 발생하지 않았을 때만 실행
        print("처리 결과:", result)
    finally:
        # 항상 실행되는 블록 (자원 정리, 로깅 등)
        print("프로세스 완료.")

# 사용 예
process_file("data.txt")


```python
코드 흐름 설명
FileNotFoundError: 파일이 존재하지 않으면 가장 먼저 잡혀서 메시지를 출력합니다.

ValueError: 파일 내용이 "123" 같은 숫자 문자열이 아니면 두 번째로 잡힙니다.

ZeroDivisionError: 읽어들인 숫자가 0이면 세 번째로 잡힙니다.

Exception: 위 세 예외 외에 다른 모든 예외를 마지막에 한 번에 처리합니다.

else / finally: else는 예외가 없을 때, finally는 예외 발생 여부와 상관없이 항상 실행됩니다.

이처럼 구체적인 예외부터 정의해야, 의도한 대로 각 예외를 세분화해서 처리할 수 있고, 마지막에 범용 예외로 “그 외”를 안전하게 받쳐줄 수 있습니다.

## finally 블록

### 예외 발생 여부와 상관없이 항상 실행

### 파일 닫기 · 연결 해제 · 트랜잭션 종료 등 정리(cleanup) 작업에 사용

## 핵심 포인트
- **SyntaxError vs Exception**
- **except 블록 순서**  
  구체적 예외 → 범용 예외 순으로 작성  
- **else**  
  예외 없이 정상 종료되었을 때만 실행  
- **finally**  
  항상 실행, 리소스 정리용  

---

## 생각하기
- **try 없이 except만 쓰면?**  
  - `SyntaxError: 'except' must be preceded by 'try'`  
  - `except`는 “어떤 try 블록에서” 발생한 예외를 잡는 역할을 함  
- **else 블록은 언제 실행되는가?**  
  - `try` 블록 안의 모든 코드가 예외 없이 성공해야만 실행  

---

## 예외 고급
### 왜 예외 고급 처리가 필요한가?
- 단순히 `print`만 하면 어디서 왜 났는지 추적하기 어려움  
- **구체적 예외 객체** 활용 시:  
  1. 오류 타입 분류 (ZeroDivisionError, KeyError 등)  
  2. 상황별 복구 로직 작성 가능  
  3. 디버깅 시 실패한 라인·이유 정확히 파악  


### `except … as e`

```python
except ValueError as e:
    # e는 ValueError 객체
    # e.args 또는 str(e)로 메시지 확인 가능
    print("값 오류:", e)


## e에 실제 예외 객체(클래스 인스턴스)가 할당

## e.args, str(e)를 통해 왜 오류가 났는지 메시지 획득


## 구체적 예외부터 작성 순서 중요

```python
try:
    ...
except Exception:
    # (1) 모든 예외 다 잡음
except ValueError:
    # (2) 절대 도달 못 함!


## **범용 예외(Exception)**를 맨 아래에 두어야 함

## 예외 구분하기 예시

```python
import json

try:
    data = json.loads(input_str)
except json.JSONDecodeError:
    print("잘못된 JSON 형식!")
except KeyError:
    print("필요한 키가 없습니다!")
except Exception:
    print("알 수 없는 오류!")


## JSON 파싱 오류, 키 조회 오류, 그 외 오류를 구체적으로 구분하여 여러분이 처리할 수 있게 작성해보세요!

In [6]:
import json

def process_user_input(input_str):
    try:
        # 1) JSON 파싱
        data = json.loads(input_str)

        # 2) 필수 키 확인
        if "age" not in data:
            raise KeyError("age 키가 없습니다")

        # 3) age 값 처리
        age = data["age"]
        if not isinstance(age, int):
            raise TypeError("age는 정수여야 합니다")

        print(f"사용자 나이: {age}")

    except json.JSONDecodeError:
        # 잘못된 JSON 형식
        print("입력된 문자열이 올바른 JSON이 아닙니다.")
        # → 사용자에게 JSON 포맷 가이드 다시 보여주기 등 복구 로직

    except KeyError as e:
        # 필요한 키가 없을 때
        print(f"키 오류: {e}")
        # → 기본값 설정 or 다시 입력 요청

    except TypeError as e:
        # 타입이 잘못됐을 때
        print(f"타입 오류: {e}")
        # → 정수로 입력하라고 안내

    except Exception as e:
        # 나머지 예외(예상치 못한 오류)
        print("알 수 없는 오류:", e)
        # → 로그에 traceback 남기기 등

    finally:
        print("입력 처리 종료")


In [10]:
# 1) 올바른 JSON & 올바른 age 타입
process_user_input('{"age": 30}')
# 출력:
# 사용자 나이: 30
# 입력 처리 종료
print('\n')
# 2) 잘못된 JSON 포맷
process_user_input('{"age": 30')  
# 출력:
# 입력된 문자열이 올바른 JSON이 아닙니다.
# 입력 처리 종료
print('\n')
# 3) 필수 키(age) 누락
process_user_input('{"name": "Alice"}')
# 출력:
# 키 오류: 'age'
# 입력 처리 종료
print('\n')
# 4) age가 정수가 아닐 때
process_user_input('{"age": "30"}')
# 출력:
# 타입 오류: age는 정수여야 합니다
# 입력 처리 종료
print('\n')
# 5) 기타 예기치 못한 오류 (예: None 전달)
process_user_input(None)
# 출력:
# 알 수 없는 오류: the JSON object must be str, bytes or bytearray, not NoneType
# 입력 처리 종료
print('\n')

사용자 나이: 30
입력 처리 종료


입력된 문자열이 올바른 JSON이 아닙니다.
입력 처리 종료


키 오류: 'age 키가 없습니다'
입력 처리 종료


타입 오류: age는 정수여야 합니다
입력 처리 종료


타입 오류: the JSON object must be str, bytes or bytearray, not NoneType
입력 처리 종료




## 왜 이렇게까지 자세하고 구체적으로 처리?

- **문제 원인 파악**  
  - `json.JSONDecodeError` → 포맷이 틀림  
  - `KeyError` → 필수 데이터 누락  
  - `TypeError` → 타입 불일치  
  - 이 정보를 통해 “사용자에게 어떤 안내를 줄지” 정확히 결정할 수 있습니다.

- **맞춤 복구 로직**  
  - JSON 오류 → “예시 JSON” 템플릿을 재전송  
  - 키 오류 → “age 값을 포함해주세요”  
  - 타입 오류 → “숫자로만 입력해주세요”

- **디버깅 편의성**  
  - 각 `except` 블록에서 `e` 객체(예외 메시지)를 로그로 남기면,  
    “어느 함수, 어떤 입력에서, 어떤 이유로” 실패했는지 나중에 확인하기 쉬워집니다.

---

## 최상위 `except Exception:`

- 모든 예상 외 오류를 한 번 더 잡아보고,  
  “앱이 꺼지지 않도록” 안전망 역할을 하지만,  
- 핵심 로직에서는 가능한 **구체적 예외**를 먼저 처리해야 혼란이 없습니다.

---

## 모든 예외 잡기

- **`except Exception:`**  
  - 파이썬의 거의 모든 표준 예외(`ValueError`, `TypeError` 등)를 잡습니다.  
  - 단, `KeyboardInterrupt`(Ctrl+C)나 `SystemExit` 등은 잡지 않습니다.

- **`except:`** (아무 타입 지정 안 함)  
  - 모든 예외(표준 예외뿐 아니라 시스템 종료, 키보드 인터럽트까지) 전부 잡습니다.

---

## 너무 넓게 잡으면 디버깅 어려워짐

1. **실제 오류 원인 가림**  
   - 버그가 생겨도 무턱대고 `except:`에서 모두 처리해 버리면,  
     “왜 실패했는지” 로그도 안 남기고 넘어갈 수 있어요.

2. **의도치 않은 예외까지 흡수**  
   - 프로그램이 중단되어야 할 심각한 상태까지 계속 실행될 수 있어,  
     뒤에서 더 큰 문제(데이터 손상 등)를 일으킬 수 있습니다.


# raise 구문

## `raise` 구문

- **`raise` 구문의 필요성과 사용 시기**  
- **왜 직접 예외를 발생시키나?**  
  - 함수 내부에서 “이 입력은 잘못됐다!”라고 판단될 때  
  - 호출 코드(함수 쓰는 쪽)에게 “적절한 처리를 맡기기 위해”

- **어떤 상황에 사용하나?**  
  - 함수의 계약(입력 타입·값 조건)을 강제할 때  
  - 하위 모듈에서 에러가 났음을 상위 로직에 알리고 싶을 때


# 파이썬 예외 고급 처리 정리

프로그램 실행 중 예상치 못한 문제가 발생했을 때, 미리 대비하여 안전하게 처리하고 디버깅을 돕기 위해 예외를 배웁니다.

---

## 핵심 포인트

1. **`except Exception as e`**: 예외 객체 `e`로 상세 메시지 활용  
2. **구체적인 예외부터**: `except` 블록은 구체적인 예외 순서대로 작성해야 합니다  
3. **`raise`**: 직접 예외를 던져 호출부에서 처리하게 할 수 있습니다  

---

## 1. 간단한 예외 발생 (`raise`)


In [14]:
def divide(a, b):
    if b == 0:
        raise ZeroDivisionError("0으로 나눌 수 없습니다")
    return a / b

In [16]:
divide(10, 0)

ZeroDivisionError: 0으로 나눌 수 없습니다

## b == 0일 때 직접 ZeroDivisionError를 발생시켜,
## 호출부에서 try/except ZeroDivisionError로 깔끔하게 처리할 수 있습니다.

In [ ]:
# 입력값 검증 예시
def check_input(x):
    if x < 0:
        raise ValueError("음수는 허용되지 않음")
    # x가 0 이상일 때만 이후 로직 실행


## x < 0일 때 ValueError를 던져 호출부에서 처리하도록 합니다.

# 예외 처리 없을 때 vs 있을 때
## 예외 처리 없을 때 (불편)

```python
def run_task():
    try:
        # 1) 설정 파일 읽기
        with open("config.json") as f:
            config = json.load(f)

        # 2) 사용자 입력 받아 나누기
        a = int(input("첫 번째 숫자: "))
        b = int(input("두 번째 숫자: "))
        result = a / b

        # 3) 결과 출력
        print("결과:", result)
    except:
        # 어떤 예외인지, 어디서 났는지 전혀 모름
        print("문제가 발생했습니다.")
    finally:
        print("입력 처리 종료")


## 오류가 어디서 났는지 모름

## config.json이 없을 때인지, int()가 실패했는지, b=0인지 구분 불가

## 개발자가 로그를 남기지 않으면 디버깅이 거의 불가능

# 예외 처리 있을 때 (편리)

In [ ]:
import json
import traceback

def run_task():
    try:
        # 1) 설정 파일 읽기
        with open("config.json") as f:
            config = json.load(f)

        # 2) 사용자 입력 받아 나누기
        a = int(input("첫 번째 숫자: "))
        b = int(input("두 번째 숫자: "))
        result = a / b

        # 3) 결과 출력
        print("결과:", result)

    except FileNotFoundError as e:
        # 설정 파일이 없을 때
        print("설정 파일을 찾을 수 없습니다:", e)
        # 복구 로직: 기본 설정 사용
        config = {"mode": "default"}
        print("기본 설정으로 계속합니다.")

    except ValueError as e:
        # 숫자 변환이 실패했을 때
        print("입력된 값이 숫자가 아닙니다:", e)
        # 복구 로직: 다시 입력받기
        return run_task()

    except ZeroDivisionError as e:
        # 0으로 나눌 때
        print("0으로 나눌 수 없습니다:", e)
        # 복구 로직: b를 1로 설정
        b = 1
        print("b=1로 설정하고 다시 시도합니다.")
        print("결과:", a / b)

    except Exception as e:
        # 나머지 모든 예외
        print("예상치 못한 오류가 발생했습니다.")
        # traceback으로 디버깅 정보 한눈에 보기
        traceback.print_exc()

    finally:
        print("작업이 종료되었습니다.")


## 예외 종류별로 복구 로직을 분리 → 사용자 경험 개선

## traceback.print_exc()로 상세 디버깅 정보 확인 가능

# 호출부에서 직접 예외 발생 처리

In [ ]:
def get_weather(city):
    if not isinstance(city, str):
        # city가 문자열이 아니면 TypeError 예외를 직접 발생시킨다
        raise TypeError("도시 이름은 문자열이어야 합니다")
    # city가 문자열일 때만 API 호출
    return fetch_weather_api(city)

try:
    info = get_weather(123)  # 정수 123을 넣었으므로
except TypeError as e:
    # raise된 TypeError가 잡혀서 이 블록이 실행된다
    print("입력 오류:", e)


In [ ]:
##

# 예외처리 실습 문제 & 퀴즈

---

## 1. 실습 문제

### 1-1. 파일 읽기 안전하게 구현하기  
**요구사항**  
- 사용자로부터 파일 경로를 입력받아 파일 내용을 출력하는 `print_file(path)` 함수를 작성하세요.  
- 파일이 존재하지 않으면 `"파일이 없습니다: <경로>"` 메시지를 출력하고 종료합니다.  
- 인코딩 에러가 발생하면 `"인코딩 오류 발생"` 메시지를 출력하도록 처리하세요.  
- 그 외 예외는 모두 `traceback.print_exc()`로 출력하세요.  
- 항상 마지막에 `"프로그램 종료"`를 출력하도록 `finally`를 활용하세요.

---

### 1-2. 숫자 계산기 예외 처리  
**요구사항**  
- 두 개의 문자열 입력을 받아 정수로 변환한 뒤 나눗셈을 수행하는 `safe_divide(a_str, b_str)` 함수를 작성하세요.  
- 다음 예외를 각각 처리하세요:  
  1. `ValueError` → `"숫자가 아닙니다: <입력값>"`  
  2. `ZeroDivisionError` → `"0으로 나눌 수 없습니다"`  
  3. 그 외 예외 → `"알 수 없는 오류 발생"`  
- 함수는 결과가 정상일 때만 나눗셈 결과를 반환하고, 예외 시에는 `None`을 반환합니다.

---

### 1-3. 사용자 정보 파싱  
**요구사항**  
- JSON 문자열로 전달된 사용자 정보를 파싱하는 `parse_user(json_str)` 함수를 작성하세요.  
- JSON 파싱 실패 시 `"JSON 형식 오류"`  
- 키 `"name"`, `"age"`가 없으면 `"필수 키 누락: <키>"`  
- `"age"`가 정수가 아니면 `"잘못된 나이 형식"`  
- 예외가 없으면 `{ "name": ..., "age": ... }` 딕셔너리를 반환하세요.

---

### 1-4. 입력 검증 함수 만들기  
**요구사항**  
- 함수 `set_volume(vol)`은 볼륨값 `vol`을 0~100 사이의 정수로 설정해야 합니다.  
- `vol`이 정수가 아니면 `TypeError("볼륨은 정수여야 합니다")`  
- 0보다 작거나 100보다 크면 `ValueError("볼륨 범위는 0~100")`  
- 정상 시 `"볼륨 설정: <vol>"` 문자열을 반환하세요.  
- 호출부에서 이 함수 호출 뒤 `try/except`로 예외 메시지를 출력하는 예시 코드를 작성하세요.

---

## 2. 퀴즈

1. **`except` 블록 순서**  
   ```python
   try:
       …
   except Exception:
       …
   except ValueError:
       …
#### 윗 코드는 잘못되었을까요?

## 3. finally 블록
#### finally 블록은 언제 실행되나요? return 문이 포함된 try 안에서도 실행될까요?

```python
def foo():
    try:
        bar()
    except ValueError:
        print("foo에서 처리")

def bar():
    raise TypeError("문제 발생")

foo()

위 코드를 실행하면 어떤 예외가 출력되나요? 왜 TypeError가 잡히지 않을까요?

In [ ]:
# 예시 답안:
def print_file(path):
    try:
        with open(path, encoding='utf-8') as f:
            print(f.read())
    except FileNotFoundError:
        print(f"파일이 없습니다: {path}")
    except UnicodeDecodeError:
        print("인코딩 오류 발생")
    except Exception:
        traceback.print_exc()
    finally:
        print("프로그램 종료")

###############################
def safe_divide(a_str, b_str):
    try:
        a = int(a_str)
        b = int(b_str)
        return a / b
    except ValueError as e:
        print(f"숫자가 아닙니다: {e.args[0].split()[-1]}")
        return None
    except ZeroDivisionError:
        print("0으로 나눌 수 없습니다")
        return None
    except Exception:
        print("알 수 없는 오류 발생")
        return None

###############################
import json

def parse_user(json_str):
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("JSON 형식 오류")
        return None

    for key in ("name", "age"):
        if key not in data:
            print(f"필수 키 누락: {key}")
            return None

    age = data["age"]
    if not isinstance(age, int):
        print("잘못된 나이 형식")
        return None

    return {"name": data["name"], "age": age}
###############################
def set_volume(vol):
    if not isinstance(vol, int):
        raise TypeError("볼륨은 정수여야 합니다")
    if vol < 0 or vol > 100:
        raise ValueError("볼륨 범위는 0~100")
    return f"볼륨 설정: {vol}"

# 호출부 예시
try:
    msg = set_volume(75)
    print(msg)
    msg = set_volume(150)
    print(msg)
except Exception as e:
    print(e)

- 첫 번째 except Exception:이 모든 예외를 잡아 버리기 때문에 뒤의 except ValueError:는 절대 실행되지 않습니다.
- 구체적 예외부터, 범용 예외는 맨 아래에 두어야 합니다.

## 1. `finally` 블록은 언제 실행되나요?

- `try`/`except` 블록 실행 흐름에 관계없이 **항상** 실행됩니다.  
- 설령 `try` 안에 `return`, `break`, `continue` 등이 있어도, 그 동작이 완료되기 **직전에** `finally` 블록이 실행됩니다.

```python
def func():
    try:
        print("try 시작")
        return "반환값"
    finally:
        print("finally 실행")

print(func())
# 출력 순서:
# try 시작
# finally 실행
# 반환값


In [23]:
def foo():
    try:
        bar()
    except ValueError:
        print("foo에서 처리")

def bar():
    raise TypeError("문제 발생")

foo()


TypeError: 문제 발생

- **왜 `TypeError`가 잡히지 않을까요?**
  - `foo()` 안의 `except` 블록은 **`ValueError`만** 처리하도록 되어 있습니다.  
  - `bar()`에서 발생한 `TypeError`는 `ValueError`가 아니므로, 해당 `except` 블록에 걸리지 않고 상위로 전파되어 프로그램이 중단됩니다.  
  - 만약 `TypeError`까지 잡으려면 다음처럼 예외 종류를 추가해야 합니다:

    ```python
    def foo():
        try:
            bar()
        except ValueError:
            print("ValueError 처리")
        except TypeError:
            print("TypeError 처리")
    ```


In [25]:
def foo():
    try:
        bar()
    except ValueError:
        print("ValueError 처리")
    except TypeError:
        print("TypeError 처리")

def bar():
    raise TypeError("문제 발생")

foo()


TypeError 처리
